In [ ]:
import subprocess
import numpy as np
import pandas as pd

algorithms = ["sha1", "rc4", "blowfish", "dsa"]

block_sizes = [16, 64, 256, 1024, 8192, 16384]

trials = 10

def run_openssl_speed(algorithm):
    
    result = subprocess.run(["openssl", "speed", algorithm], capture_output=True, text=True)
    return result.stdout

def is_number(s):
    try:
        float(s.strip('k'))
        return True
    except ValueError:
        return False

def extract_data(algorithm, output):
    lines = output.splitlines()
    data = []
    for line in lines:
        if algorithm in line:
            values = line.split()
            
            speed_values = [float(val.strip('k')) for val in values[1:] if is_number(val)]
            if speed_values:  
                data.append(speed_values)
    return np.array(data)

results = {alg: [] for alg in algorithms}

for trial in range(trials):
    for algorithm in algorithms:
        print(f"Running trial {trial + 1} for {algorithm}...")
        output = run_openssl_speed(algorithm)
        data = extract_data(algorithm, output)
        if len(data) > 0:
            results[algorithm].append(data[0])

averages = {alg: np.mean(results[alg], axis=0) if len(results[alg]) > 0 else [] for alg in algorithms}
std_devs = {alg: np.std(results[alg], axis=0) if len(results[alg]) > 0 else [] for alg in algorithms}

final_data = []
for algorithm in algorithms:
    available_block_sizes = min(len(averages[algorithm]), len(block_sizes))  
    for i in range(available_block_sizes):
        final_data.append([algorithm, block_sizes[i], averages[algorithm][i], std_devs[algorithm][i]])

df = pd.DataFrame(final_data, columns=["Algorithm", "Block Size (bytes)", "Average (kB/sec)", "Std Dev (kB/sec)"])

print(df)

df.to_csv("openssl_speed_results.csv", index=False)


In [ ]:
import subprocess
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

algorithms = ["sha1", "rc4", "blowfish", "dsa"]

block_sizes = [16, 64, 256, 1024, 8192, 16384]

trials = 10

def run_openssl_speed(algorithm):
    result = subprocess.run(
        ["openssl", "speed", "-evp", algorithm, "-provider", "legacy"], 
        capture_output=True, text=True
    )
    return result.stdout


def is_number(s):
    try:
        float(s.strip('k'))
        return True
    except ValueError:
        return False

def extract_data(algorithm, output):
    lines = output.splitlines()
    data = []
    for line in lines:
        if algorithm in line:
            values = line.split()
            
            speed_values = [float(val.strip('k')) for val in values[1:] if is_number(val)]
            if speed_values:  
                data.append(speed_values)
    return np.array(data)

def run_trial(algorithm):
    results = []
    for trial in range(trials):
        print(f"Running trial {trial + 1} for {algorithm}...")
        output = run_openssl_speed(algorithm)
        data = extract_data(algorithm, output)
        if len(data) > 0:
            results.append(data[0])
    return results

def run_all_trials_parallel():
    results = {alg: [] for alg in algorithms}
    
    with ThreadPoolExecutor() as executor:
        future_to_alg = {executor.submit(run_trial, algorithm): algorithm for algorithm in algorithms}

        for future in future_to_alg:
            algorithm = future_to_alg[future]
            try:
                data = future.result()
                results[algorithm] = data
            except Exception as exc:
                print(f"{algorithm} generated an exception: {exc}")
    
    return results

results = run_all_trials_parallel()

averages = {alg: np.mean(results[alg], axis=0) if len(results[alg]) > 0 else [] for alg in algorithms}
std_devs = {alg: np.std(results[alg], axis=0) if len(results[alg]) > 0 else [] for alg in algorithms}

final_data = []
for algorithm in algorithms:
    available_block_sizes = min(len(averages[algorithm]), len(block_sizes))  
    for i in range(available_block_sizes):
        final_data.append([algorithm, block_sizes[i], averages[algorithm][i], std_devs[algorithm][i]])

df = pd.DataFrame(final_data, columns=["Algorithm", "Block Size (bytes)", "Average (kB/sec)", "Std Dev (kB/sec)"])

print(df)

df.to_csv("openssl_speed_results.csv", index=False)

In [ ]:
import subprocess
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm  

algorithms = ["sha1", "rc4", "blowfish", "dsa"]

block_sizes = [16, 64, 256, 1024, 8192, 16384]

trials = 10

def run_openssl_speed(algorithm):
    result = subprocess.run(
        ["openssl", "speed", "-evp", algorithm, "-provider", "legacy"], 
        capture_output=True, text=True
    )
    return result.stdout


def is_number(s):
    try:
        float(s.strip('k'))
        return True
    except ValueError:
        return False

def extract_data(algorithm, output):
    lines = output.splitlines()
    data = []
    for line in lines:
        if algorithm in line:
            values = line.split()
            
            speed_values = [float(val.strip('k')) for val in values[1:] if is_number(val)]
            if speed_values:  
                data.append(speed_values)
    return np.array(data)

def run_trial(algorithm):
    results = []
    
    for trial in tqdm(range(trials), desc=f"Running trials for {algorithm}", leave=False):
        output = run_openssl_speed(algorithm)
        data = extract_data(algorithm, output)
        if len(data) > 0:
            results.append(data[0])
    return results

def run_all_trials_parallel():
    results = {alg: [] for alg in algorithms}
    
    
    with ThreadPoolExecutor() as executor:
        future_to_alg = {executor.submit(run_trial, algorithm): algorithm for algorithm in algorithms}

        for future in tqdm(future_to_alg, desc="Processing algorithms"):
            algorithm = future_to_alg[future]
            try:
                data = future.result()
                results[algorithm] = data
            except Exception as exc:
                print(f"{algorithm} generated an exception: {exc}")
    
    return results

results = run_all_trials_parallel()

averages = {alg: np.mean(results[alg], axis=0) if len(results[alg]) > 0 else [] for alg in algorithms}
std_devs = {alg: np.std(results[alg], axis=0) if len(results[alg]) > 0 else [] for alg in algorithms}

final_data = []
for algorithm in algorithms:
    available_block_sizes = min(len(averages[algorithm]), len(block_sizes))  
    for i in range(available_block_sizes):
        final_data.append([algorithm, block_sizes[i], averages[algorithm][i], std_devs[algorithm][i]])

df = pd.DataFrame(final_data, columns=["Algorithm", "Block Size (bytes)", "Average (kB/sec)", "Std Dev (kB/sec)"])

print(df)

df.to_csv("openssl_speed_results.csv", index=False)


In [2]:
!pip install prettytable

In [6]:
import os
import time
import subprocess
import statistics
from tempfile import NamedTemporaryFile
from prettytable import PrettyTable

FILE_SIZE = 1024 * 1024 * 1024  # 1GB
RUNS = 10

def run_openssl_command(command):
    start_time = time.time()
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    process.communicate()
    end_time = time.time()
    return end_time - start_time

def run_command_multiple_times(command, runs):
    times = []
    for _ in range(runs):
        execution_time = run_openssl_command(command)
        times.append(execution_time)
    return times

def main():
    with NamedTemporaryFile(delete=False) as temp_file:
        temp_filename = temp_file.name
        temp_file.write(os.urandom(FILE_SIZE))

    dsa_param_file = NamedTemporaryFile(delete=False)
    dsa_private_key_file = NamedTemporaryFile(delete=False)
    dsa_public_key_file = NamedTemporaryFile(delete=False)

    run_openssl_command(f"openssl dsaparam 1024 -out {dsa_param_file.name}")
    run_openssl_command(f"openssl gendsa {dsa_param_file.name} -out {dsa_private_key_file.name}")
    run_openssl_command(f"openssl dsa -in {dsa_private_key_file.name} -pubout -out {dsa_public_key_file.name}")

    commands = {
        "SHA1": f"openssl dgst -sha1 -out /dev/null {temp_filename}",
        "RC4": f"openssl enc -rc4 -nosalt -k secret -nopad -in {temp_filename} -out /dev/null",
        "Blowfish": f"openssl enc -bf -nosalt -k secret -nopad -in {temp_filename} -out /dev/null",
        "DSA": f"openssl dgst -dss1 -sign {dsa_private_key_file.name} -out /dev/null {temp_filename}"
    }

    results = PrettyTable()
    results.field_names = ["Algorithm", "Average Time (s)", "Std Dev (s)"]

    for algo, command in commands.items():
        print(f"Running {algo}...")
        times = run_command_multiple_times(command, RUNS)
        avg_time = statistics.mean(times)
        std_dev = statistics.stdev(times) if len(times) > 1 else 0
        results.add_row([algo, f"{avg_time:.4f}", f"{std_dev:.4f}"])

    print("\nResults:")
    print(results)

    os.unlink(temp_filename)
    os.unlink(dsa_param_file.name)
    os.unlink(dsa_private_key_file.name)
    os.unlink(dsa_public_key_file.name)

if __name__ == "__main__":
    main()

Running SHA1...
Running RC4...
Running Blowfish...
Running DSA...

Results:
+-----------+------------------+-------------+
| Algorithm | Average Time (s) | Std Dev (s) |
+-----------+------------------+-------------+
|    SHA1   |      0.4779      |    0.0106   |
|    RC4    |      0.0030      |    0.0022   |
|  Blowfish |      0.0022      |    0.0001   |
|    DSA    |      0.0020      |    0.0001   |
+-----------+------------------+-------------+


In [1]:
import os
import time
import subprocess
import statistics
import threading
from tempfile import NamedTemporaryFile
from prettytable import PrettyTable
from concurrent.futures import ThreadPoolExecutor, as_completed

FILE_SIZE = 1024 * 1024 * 1024  # 1GB
RUNS = 100
MAX_WORKERS = 16

def run_openssl_command(command):
    start_time = time.time()
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    process.communicate()
    end_time = time.time()
    return end_time - start_time

def run_command_multiple_times(command, runs):
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(run_openssl_command, command) for _ in range(runs)]
        return [future.result() for future in as_completed(futures)]

def main():
    try:
        with NamedTemporaryFile(delete=False) as temp_file:
            temp_filename = temp_file.name
            temp_file.write(os.urandom(FILE_SIZE))

        dsa_param_file = NamedTemporaryFile(delete=False)
        dsa_private_key_file = NamedTemporaryFile(delete=False)
        dsa_public_key_file = NamedTemporaryFile(delete=False)

        try:
            run_openssl_command(f"openssl dsaparam 1024 -out {dsa_param_file.name}")
            run_openssl_command(f"openssl gendsa {dsa_param_file.name} -out {dsa_private_key_file.name}")
            run_openssl_command(f"openssl dsa -in {dsa_private_key_file.name} -pubout -out {dsa_public_key_file.name}")
            
            commands = {
                "SHA1": f"openssl dgst -sha1 -out /dev/null {temp_filename}",
                "RC4": f"openssl enc -rc4 -nosalt -k secret -nopad -in {temp_filename} -out /dev/null",
                "Blowfish": f"openssl enc -bf -nosalt -k secret -nopad -in {temp_filename} -out /dev/null",
                "DSA": f"openssl dgst -dss1 -sign {dsa_private_key_file.name} -out /dev/null {temp_filename}"
            }

            results = PrettyTable()
            results.field_names = ["Algorithm", "Average Time (s)", "Std Dev (s)"]

            for algo, command in commands.items():
                print(f"Running {algo}...")
                times = run_command_multiple_times(command, RUNS)
                avg_time = statistics.mean(times)
                std_dev = statistics.stdev(times) if len(times) > 1 else 0
                results.add_row([algo, f"{avg_time:.4f}", f"{std_dev:.4f}"])

            print("\nResults:")
            print(results)

        finally:
            for file in [dsa_param_file, dsa_private_key_file, dsa_public_key_file]:
                file.close()
                os.unlink(file.name)

    finally:
        os.unlink(temp_filename)

if __name__ == "__main__":
    main()

Running SHA1...
Running RC4...
Running Blowfish...
Running DSA...

Results:
+-----------+------------------+-------------+
| Algorithm | Average Time (s) | Std Dev (s) |
+-----------+------------------+-------------+
|    SHA1   |      0.6653      |    0.0472   |
|    RC4    |      0.0092      |    0.0101   |
|  Blowfish |      0.0067      |    0.0019   |
|    DSA    |      0.0067      |    0.0018   |
+-----------+------------------+-------------+
